In [1]:
import os
import sys
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sys.path.append("../")
from src.utils import calc_wap, calc_wap2, log_return, realized_volatility

In [3]:
INPUT_DIR = "../input"

In [4]:
book_train = pd.read_parquet(os.path.join(INPUT_DIR, "book_train.parquet/stock_id=15"))
book_train.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,0.999519,0.999839,0.999454,0.999904,2,166,2,12
1,5,1,0.999711,1.000225,0.999647,1.000289,100,20,100,20
2,5,2,0.999775,1.000225,0.999711,1.000289,1,20,400,20
3,5,3,0.999839,1.000225,0.999775,1.000289,100,20,1,20
4,5,4,0.999839,1.000225,0.999711,1.000289,1,20,400,20


In [11]:
def preprocessor_book(file_path):
    '''
    bookデータの特徴量を生成
    'log_return_realized_volatility', 'log_return2_realized_volatility',
       'wap_balance_mean', 'price_spread_mean', 'bid_spread_mean',
       'ask_spread_mean', 'volume_imbalance_mean', 'total_volume_mean',
       'wap_mean', 'log_return_realized_volatility_300',
       'log_return2_realized_volatility_300', 'wap_balance_mean_300',
       'price_spread_mean_300', 'bid_spread_mean_300', 'ask_spread_mean_300',
       'volume_imbalance_mean_300', 'total_volume_mean_300', 'wap_mean_300',
       'row_id''log_return_realized_volatility', 'log_return2_realized_volatility',
       'wap_balance_mean', 'price_spread_mean', 'bid_spread_mean',
       'ask_spread_mean', 'volume_imbalance_mean', 'total_volume_mean',
       'wap_mean', 'log_return_realized_volatility_300',
       'log_return2_realized_volatility_300', 'wap_balance_mean_300',
       'price_spread_mean_300', 'bid_spread_mean_300', 'ask_spread_mean_300',
       'volume_imbalance_mean_300', 'total_volume_mean_300', 'wap_mean_300',
       'row_id'
    '''
    df = pd.read_parquet(file_path)
    # calculate return etc
    df["wap"] = calc_wap(df)
    df["log_return"] = df.groupby("time_id")["wap"].apply(log_return)

    df["wap2"] = calc_wap2(df)
    df["log_return2"] = df.groupby("time_id")["wap2"].apply(log_return)

    df["wap_balance"] = abs(df["wap"] - df["wap2"])

    df["price_spread"] = (df["ask_price1"] - df["bid_price1"]) / (
        (df["ask_price1"] + df["bid_price1"]) / 2
    )
    df["bid_spread"] = df["bid_price1"] - df["bid_price2"]
    df["ask_spread"] = df["ask_price1"] - df["ask_price2"]
    df["total_volume"] = (df["ask_size1"] + df["ask_size2"]) + (
        df["bid_size1"] + df["bid_size2"]
    )
    df["volume_imbalance"] = abs(
        (df["ask_size1"] + df["ask_size2"]) - (df["bid_size1"] + df["bid_size2"])
    )

    # dict for aggregate
    create_feature_dict = {
        "log_return": [realized_volatility],
        "log_return2": [realized_volatility],
        "wap_balance": [np.mean],
        "price_spread": [np.mean],
        "bid_spread": [np.mean],
        "ask_spread": [np.mean],
        "volume_imbalance": [np.mean],
        "total_volume": [np.mean],
        "wap": [np.mean],
    }

    #####groupby / all seconds
    df_feature = pd.DataFrame(
        df.groupby(["time_id"]).agg(create_feature_dict)
    ).reset_index()

    df_feature.columns = [
        "_".join(col) for col in df_feature.columns
    ]  # time_id is changed to time_id_

    ######groupby / last XX seconds
    last_seconds = [300]

    for second in last_seconds:
        second = 600 - second

        df_feature_sec = pd.DataFrame(
            df.query(f"seconds_in_bucket >= {second}")
            .groupby(["time_id"])
            .agg(create_feature_dict)
        ).reset_index()

        df_feature_sec.columns = [
            "_".join(col) for col in df_feature_sec.columns
        ]  # time_id is changed to time_id_

        df_feature_sec = df_feature_sec.add_suffix("_" + str(second))

        df_feature = pd.merge(
            df_feature,
            df_feature_sec,
            how="left",
            left_on="time_id_",
            right_on=f"time_id__{second}",
        )
        df_feature = df_feature.drop([f"time_id__{second}"], axis=1)

    # create row_id
    stock_id = file_path.split("=")[1]
    df_feature["row_id"] = df_feature["time_id_"].apply(lambda x: f"{stock_id}-{x}")
    df_feature = df_feature.drop(["time_id_"], axis=1)

    return df_feature

In [13]:
%%time
file_path = os.path.join(INPUT_DIR, "book_train.parquet/stock_id=0")
df = preprocessor_book(file_path)
df.columns

CPU times: user 3.96 s, sys: 35.9 ms, total: 4 s
Wall time: 3.99 s


Index(['log_return_realized_volatility', 'log_return2_realized_volatility',
       'wap_balance_mean', 'price_spread_mean', 'bid_spread_mean',
       'ask_spread_mean', 'volume_imbalance_mean', 'total_volume_mean',
       'wap_mean', 'log_return_realized_volatility_300',
       'log_return2_realized_volatility_300', 'wap_balance_mean_300',
       'price_spread_mean_300', 'bid_spread_mean_300', 'ask_spread_mean_300',
       'volume_imbalance_mean_300', 'total_volume_mean_300', 'wap_mean_300',
       'row_id'],
      dtype='object')